In [1]:
from collections import namedtuple
from enum import Enum
from typing import Dict, List

with open("input.txt") as f:
    lines = [l.strip() for l in f.readlines()]

In [2]:
Point = namedtuple("Point", "x y")

class Element(Enum):
    ROCK = 1
    WATER = 2

def line_to_points(l: str) -> List[Point]:
    """Parses a line in the format '498,4 -> 498,6 -> 496,6' into list of points"""
    parts = l.split("->")
    points = []
    for part in parts:
        x, y = part.split(",")
        points.append(Point(int(x), int(y)))
    return points

def all_points_inclusive(start, end: Point) -> List[Point]:
    """Take start and end point, return all points between them with start and end inclusive"""
    points = []
    if start.x == end.x:
        min_y, max_y = min(start.y, end.y), max(start.y, end.y)
        return [
            Point(start.x, y) for y in range(min_y, max_y+1)
        ]

    min_x, max_x = min(start.x, end.x), max(start.x, end.x)
    return [
        Point(x, start.y) for x in range(min_x, max_x+1)
    ]
 

def init_grid(lines: List[str]) -> Dict[Point, Element]:
    grid = {}
    for l in lines:
        parsed = line_to_points(l)
        for i in range(len(parsed)-1):
            points = all_points_inclusive(parsed[i], parsed[i+1])
            
            for point in points:
                grid[point] = Element.ROCK
    
    return grid

In [3]:
def simulate(grid: Dict[Point, Element]) -> Dict[Point, Element]:
    """Run simulation until water overflows"""
    # TODO: find lowest Y point, and once we have a water drop that
    # passes the lowest Y point, then we are done
    lowest_y = float("-inf")
    for p in grid.keys():
        lowest_y = max(lowest_y, p.y)

    while True:
        # start drop at (500, 0) and apply rules until it comes to rest or passes lowest_y
        p = Point(500, 0)
        while True:
            if p.y > lowest_y:
                return grid

            if Point(p.x, p.y+1) not in grid:
                p = Point(p.x, p.y+1)
            elif Point(p.x-1, p.y+1) not in grid:
                p = Point(p.x-1, p.y+1)
            elif Point(p.x+1, p.y+1) not in grid:
                p = Point(p.x+1, p.y+1)
            else:
                grid[p] = Element.WATER
                break


num_water = sum(el == Element.WATER for el in simulate(init_grid(lines)).values())
f"part 1: {num_water}"

'part 1: 892'